In [196]:
import pandas as pd 
import numpy as np
from datetime import datetime



In [197]:
def ohe_mail_type (df):
    unique_mail_types = df['mail_type'].apply(lambda x: x.split('/')).explode().unique()
    for mt in unique_mail_types:
        df[f'mail_type_{mt}'] = df['mail_type'].apply(lambda x: 1 if mt in x else 0)
    df.drop('mail_type',axis =1,inplace = True)
    return(df)


def data_cleaner (df, chars_in_subject = 42 ):
    df['org'] = df['org'].fillna('org_undefinded')
    df['tld'] = df['tld'].fillna('tld_undefinded')
    df['mail_type'] = df['mail_type'].fillna('mail_type_undefinded').apply(lambda x:x.lower())
    df['chars_in_subject'] = df['chars_in_subject'].fillna(chars_in_subject)
    df.drop([ 'date','tld'],axis= 1, inplace=True)
    return (df)

In [198]:
train_df = pd.read_csv('train.csv', )
test_df = pd.read_csv('test.csv', )
# train_df.drop('Unnamed: 0',axis=1,inplace = True)
# test_df.drop('Unnamed: 0',axis=1,inplace = True)
test_df['label'] = 'test_df'

In [199]:
df = train_df.append(test_df, ignore_index=True)
df[df ['Unnamed: 0'] == 40]


C:\Users\arind\AppData\Local\Temp\ipykernel_20172\170277492.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train_df.append(test_df, ignore_index=True)


,Unnamed: 0,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
40,40,"Fri, 07 Apr 2017 15:30:01 +0530",nrsc,gov.in,0,0,multipart/alternative,0,4,1,0,32.0,1210,0
80216,40,"Sat, 24 Mar 2018 21:53:50 +0530",iiitd,ac.in,1,0,multipart/alternative,1,7,1,0,30.0,5399,test_df


In [200]:
df['Ratio-SubtoBody'] = df['chars_in_body'] / (df['chars_in_subject'] + df['chars_in_body'])

In [201]:
import datefinder
# df['day'] = df['date'].apply(lambda x:x.split(',')[0])
def find_date(x):
    for x in datefinder.find_dates(x):
        return(x)
        break

df['date'] = df['date'].apply(find_date)

In [202]:
# day_labeldf = df.groupby(['label','day']).size().reset_index()

In [203]:
df['date'] = pd.to_datetime(df['date'], utc = True)

In [204]:
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['weekday'] = pd.DatetimeIndex(df['date']).dayofweek
df['hour'] = pd.DatetimeIndex(df['date']).hour
df.drop('date',axis=1,inplace=True)

In [205]:
df['tld'] = df['tld'].fillna('com')

In [209]:
df['tld_back'] = df['tld'].apply(lambda x : x.split('.')[-1] if '.' in x else x ).apply(lambda x : x.lower())
df['tld_front'] = df['tld'].apply(lambda x : x.split('.')[0] if '.' in x else 'not_found' ).apply(lambda x : x.lower())
df['mail_type'] = df['mail_type'].apply(lambda x : str(x).lower()).fillna('multipart/alternative')

In [210]:
df

,Unnamed: 0,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,Ratio-SubtoBody,year,month,weekday,hour,tld_back,tld_front
0,0,reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2,0.999388,2017,11,0,10,in,ebay
1,1,edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1,0.965124,2018,2,2,11,com,efinmail
2,2,usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1,0.998610,2016,7,2,19,com,not_found
3,3,granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1,0.999976,2019,10,4,9,ai,not_found
4,4,github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1,0.979617,2017,11,1,11,com,not_found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114536,34360,quora,com,0,0,multipart/alternative,8,110,1,1,134.0,89070,test_df,0.998498,2016,9,1,2,com,not_found
114537,34361,shop,sdconnect.in,0,0,multipart/alternative,23,117,1,0,52.0,24241,test_df,0.997859,2016,8,4,6,in,sdconnect
114538,34362,iiitd,ac.in,0,0,multipart/alternative,1,4,1,0,35.0,4558,test_df,0.992380,2016,5,2,11,in,ac
114539,34363,tatadocomo,com,0,0,multipart/alternative,0,6,1,0,38.0,2591,test_df,0.985546,2014,12,0,9,com,not_found


In [213]:
df.groupby(['label','tld_front']).size().to_csv('label-tldfr.csv')

In [135]:
df.groupby(['label','month']).size().to_csv('label-month.csv')

In [136]:
df.groupby(['label','year']).size().to_csv('label-yr.csv')

In [139]:
df.groupby(['label','hour']).size().to_csv('label-hr.csv')

In [145]:
list(df['org'].unique())

['reply',
 'edm',
 'usebackpack',
 'granular',
 'github',
 'iiitd',
 'piazza',
 'in',
 'redwolf',
 'm',
 'mailer',
 'kaggle',
 'google',
 'coursera',
 'linkedin',
 'twitter',
 'sbi',
 'academia-mail',
 'topcoder',
 nan,
 'nrsc',
 'atcoder',
 'email-marriott',
 'e',
 'amazon',
 'zoom',
 'marketing',
 'glassdoor',
 'primevideo',
 'quora',
 'arya',
 'facebookmail',
 'gmail',
 'ZOONIVERSE',
 'slack',
 'walkfree',
 'emailer',
 'airvistara',
 'localcirclesmail',
 'ieee',
 'sampark',
 'digital',
 'email',
 'asuswebstorage',
 'mail',
 'phpclasses',
 'nvidia',
 'centralesupelec',
 'innerchef',
 'youtube',
 'technophilia',
 'manuscriptcentral',
 'eos',
 'shining3d',
 'stockroom',
 'edx',
 'accounts',
 'flipkartpromotions',
 'se',
 'visale',
 'freelancer',
 'hackerearth',
 'imindmap',
 'freecharge-offers',
 'oracle-mail',
 'mentor',
 'zen-makemytrip',
 'udacity',
 'dropboxmail',
 'careernet',
 'angel',
 'applemusic',
 'travel',
 'flipkartletters',
 'monsterindia',
 'olacabs',
 'standupalice',
 'c